
# Modeling method of Fluid-structure interaction system based on deep neural network

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Overview

Aeroelastic problem of aircraft is a typical Fluid–structure interaction (FSI) problem, which studies the coupling relationship between aircraft structure and aerodynamic force. High accuracy Computational fluid dynamics (CFD) technology can accurately simulate the evolution process of the flow field around the structure to obtain the force situation of the structure, but the huge number of grids leads to high computing costs. Many researchers try to use the data-driven method to build the flow field evolution model to achieve rapid prediction of the flow field with high accuracy, so as to improve the simulation efficiency of the Fluid–structure interaction system. In recent years, the rapidly developing deep neural network technology relies on its powerful nonlinear learning ability and deep feature capture ability, and has achieved many successful applications in flow field modeling problems. Among them, flow field reconstruction achieves rapid prediction of different flow fields by constructing a mapping model between geometric shapes and flow conditions to flow field information at spatial points, which is highly concerned for its ability to quickly provide the current flow field state.

In order to efficiently solve the flow field reconstruction of the Fluid–structure interaction problem, this paper coupled the neural network model with the computational structural dynamic equation, realized the modeling of the Fluid–structure interaction system, further improved the neural network structure, optimized the data structure, so as to obtain more accurate flow field prediction results and achieve more accurate Fluid–structure interaction response prediction.

## Problem description

The traditional Fluid–structure interaction numerical simulation framework consists of a Computational fluid dynamics solver and a computational Solid mechanics solver. The two solvers solve the state of the fluid and structure at the next moment in the fluid domain and the solid domain respectively, and transmit information at the interface as the input for the next calculation. The coupling process is shown in the following figure. The Fluid–structure interaction modeling framework based on the depth neural network proposed in this paper still uses the same strategy. The framework uses the depth neural network instead of the CFD solver to predict the flow field evolution. The structural response is still calculated by the CSD solver. The structural displacement and flow field surface pressure are transferred between the depth neural network and the computational Solid mechanics solver.

## Technology path

The specific process of mindflow to solve this problem is as follows:

1.Create data sets based on CFD numerical simulation results.

2.The model is built using mindspire deep learning framework.

3.Define the optimizer and loss function.

4.Use mindspire's instant compilation to accelerate model training.

5.Use the trained model for reasoning and visualization.

![p1.png](./images/p1.png)

## Model Architecture

The basic framework of HDNN consists of convolutional neural network (CNN), convolutional long short-term memory network (ConvLSTM) and deconvolution neural network (DeCNN). CNN reduces the dimensionality of the time series flow field and achieves feature extraction; ConvLSTM learns low dimensional spatiotemporal features and makes predictions; Finally, DeCNN achieves reconstruction of predicted flow fields

+ Input layer: current flow field state and boundary conditions
+ Convolutional layer: Capturing the spatial features of the flow field and reducing its dimensionality, using low dimensional flow field features to predict flow field evolution can improve computational efficiency
+ LSTM layer: predicts the flow field characteristics of the next moment based on the captured current flow field characteristics and structural motion conditions
+ Deconvolution output layer: Restores the low-dimensional features of the predicted flow field to high-dimensional space, reconstructs the transient flow field at the next moment through multi-layer DeCNN, and outputs visual prediction results

![HDNN.jpg](./images/HDNN.jpg)

## Training dataset

The dataset is constructed from multidimensional matrix flow field snapshot matrix constructed from numerical simulation of unsteady two-dimensional cylindrical flow field data

+ The moving structure (cylinder) in the flow field makes one-dimensional Simple harmonic motion in the vertical direction. Physical modeling of two-dimensional cylindrical flow field, mesh discretization/partitioning, and solving control equations using Reynolds time averaged simulation method to obtain flow field information. Dimensionalize the physical quantities of the flow field and place grid sampling points in the sampling area to obtain a sample set for training and testing
+ Each flow field snapshot contains three channels, representing the pressure distribution information, horizontal velocity information, and vertical velocity information of the flow field
+ Dataset:[Download location](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/fluid_structure_interaction/)

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, data_sink, jit
from mindflow.utils import load_yaml_config

from src import generate_dataset, AEnet, save_loss_curve

## Training environment

+ The training adopts the static graphical model of Mindspot framework (GRAPH)
+ Train on CPU, GPU, or Ascend

In [2]:
set_seed(0)
np.random.seed(0)

## Training hyperparameter

Obtain hyperparameters for models, data, and optimizers from config

In [3]:
parser = argparse.ArgumentParser(description="cylinder around flow ROM")

parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./summary")
parser.add_argument("--device_target", type=str, default="Ascend", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'GPU','Ascend'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--data_list", type=list, default=['5.0', '5.5', '6.0', '6.5'], help="The type for training")
parser.add_argument('--batch_size', type=int, default=32, help="batch size")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")

args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs, save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target, device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

config = load_yaml_config(args.config_file_path)
data_params = config["data"]
model_params = config["model"]
optimizer_params = config["optimizer"]

## Training process file save path

Save the trained model file in a folder every certain number of training sessions

In [4]:
ckpt_dir = optimizer_params["ckpt_dir"]
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## Constructing neural network and optimizer

The convolutional layer of the neural network has a total of 12 layers, ConvLSTM has 1 layer, and deconvolution has a total of 12 layers

The Loss function uses the Mean squared error Loss function, and the optimizer uses the Adam (Adaptive Moment Estimation) optimization algorithm

In [5]:
model = AEnet(in_channels=model_params["in_channels"],
              num_layers=model_params["num_layers"],
              kernel_size=model_params["kernel_size"],
              num_convlstm_layers=model_params["num_convlstm_layers"])

loss_func = nn.MSELoss()
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=optimizer_params["lr"])
if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, 'O1')
else:
    loss_scaler = None

## Training framework

Define the forward propagation function forward_ Fn, compare the predicted value with the true value to obtain the loss value and return it

In [6]:
def forward_fn(inputs, velocity, ur, label):
    pred = model(inputs, velocity, ur)
    loss = loss_func(pred, label)

    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

## Dataset loading

To generate_dataset parameter transfer to obtain training and validation datasets

In [7]:
print(f"==================Load data sample ===================")
dataset_train, dataset_eval = generate_dataset(data_params["data_dir"],
                                               data_params["time_steps"],
                                               args.data_list)
print(f"======================End Load========================\n")

## Data sink and model training

Define train_ Step and Eval_ Step and use data_ Sink acceleration training, output the loss value and usage time during the training process, and save the model file every certain training round

In [8]:
print(f"====================Start train=======================")
@jit
def train_step(inputs, velocity, ur, label):
    loss, grads = grad_fn(inputs, velocity, ur, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, optimizer(grads))
    return loss

@jit
def eval_step(inputs, velocity, ur, label):
    loss = forward_fn(inputs, velocity, ur, label)
    loss = ops.sqrt(loss)
    return loss

train_sink_process = data_sink(train_step, dataset_train, sink_size=1)
eval_sink_process = data_sink(eval_step, dataset_eval, sink_size=1)
train_data_size, eval_data_size = dataset_train.get_dataset_size(), dataset_eval.get_dataset_size()

avg_train_losses = []
avg_valid_losses = []

for epoch in range(1, optimizer_params["epochs"] + 1):
    train_losses = 0
    valid_losses = 0

    local_time_beg = time.time()
    model.set_train(True)

    for _ in range(train_data_size):
        step_train_loss = ops.squeeze(train_sink_process(), axis=())
        step_train_loss = step_train_loss.asnumpy().item()
        train_losses += step_train_loss

    train_loss = train_losses / train_data_size
    avg_train_losses.append(train_loss)

    print(f"epoch: {epoch}, epoch average train loss: {train_loss :.6f}, "
          f"epoch time: {(time.time() - local_time_beg):.2f}s")

    if epoch % optimizer_params["eval_interval"] == 0:
        print(f"=================Start Evaluation=====================")

        eval_time_beg = time.time()
        model.set_train(False)
        for _ in range(eval_data_size):
            step_eval_loss = ops.squeeze(eval_sink_process(), axis=())
            step_eval_loss = step_eval_loss.asnumpy().item()
            valid_losses += step_eval_loss

        valid_loss = valid_losses / eval_data_size
        avg_valid_losses.append(valid_loss)

        print(f"epoch: {epoch}, epoch average valid loss: {valid_loss :.6f}, "
              f"epoch time: {(time.time() - eval_time_beg):.2f}s")
        print(f"==================End Evaluation======================")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/net_{epoch}.ckpt")

save_loss_curve(avg_train_losses, 'Epoch', 'avg_train_losses', 'Avg_train_losses Curve', 'Avg_train_losses.png')
save_loss_curve(avg_valid_losses, 'Epoch', 'avg_valid_losses', 'Avg_valid_losses Curve', 'Avg_valid_losses.png')

print(f"=====================End train========================")

## Set training conditions for parameter transmission

When running the file, pass in the necessary parameters through the parameter parser to start training, and print the process and device id, as well as the total training time

In [9]:
if __name__ == "__main__":
    print("Process ID:", os.getpid())
    print(f"device id: {args.device_id}")
    start_time = time.time()
    train()
    print(f"End-to-End total time: {(time.time() - start_time):.2f}s")

## Visualization of predicted flow field results

+ Moving boundary flow field prediction starts by executing eval.py. The coupled model can complete the prediction task of the entire Fluid–structure interaction evolution process under the condition that only the initial flow field state and cylinder position are given
+ The following figure shows the flow field prediction status of a fully trained HDNN model for a deep neural network at different times within a cycle

![pred_cycle_puv.jpg](./images/pred_cycle_puv.jpg)